Introduction
============

Recently, I started using the [pandas](http://pandas.pydata.org/) python library to improve the quality 
(and quantity) of statistics in my applications. One method that has helped me much is the `pivot_table`
command. This is a really (at least for me) complicated command that has
[very poor documentation](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.pivot_table.html).
Beyond this, this command is explained a little more in an article about 
[data reshaping](http://pandas.pydata.org/pandas-docs/stable/reshaping.html), however, even this leaves
much to be desired (when I first tried reading it I was overwhelmed by the amount of information there).

A great introduction to pandas is the [three part series](http://www.gregreda.com/2013/10/26/intro-to-pandas-data-structures/) by Greg Reda - it touches `pivot_table` however I was only able to understand what it happend *after* I loose a lot of time playing myself with `pivot_table`.

I have to mention that I am no expert in statistics or
numeric analysis so this post won't have any advanced 
information and may even point out some obvious things. However
keep in mind things that may seem obvious to some
experts are really difficult to grasp for a non-expert.

Before continuing, please notice that this article has been written as a [jupyter notebook](http://jupyter.org/) and was integrated with pelican using the [pelican-ipynb plugin](https://github.com/danielfrg/pelican-ipynb). I had to do some modifications to my theme to better integrate the notebook styling, however some stuff may not look as nice as the other articles.

The DataFrame
=============

The most important (new) data structure that pandas uses is the [DataFrame](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html). This can be thought as a two dimensional array, something like an Excel sheet. In the pandas nomenclature, the rows of that two-dimensional array are called *indexes* (while the columns keep their name). Also, if we horizontally/vertically pick the values of a single row(index)/column we'll be left with a different data structure called [Series](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.html) - so an array is a series. There's also a [Panel](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Panel.html) data structure which is 3-dimensional, more like a complete Excel workbook (the third dimension being the sheets of the workbook) but I won't cover that here.

More info on the above can be found on the corresponding [article about data structures](http://pandas.pydata.org/pandas-docs/stable/dsintro.html).

There are various ways to read the data for a Series or DataFrame: Initializing through arrays or dicts, reading from csv, xls, database, combinining series to create an array and various others. I won't go into any details about this but will include some examples on how to create Series and DataFrames.

Before continuing with more info on the `pivot_table` operation, I will list a number of examples as a quick reference on how to work with Series and DataFrames that will be useful in the rest of the article (and should be useful in general). Notice that I'm using pandas 0.18.1.

Using Series
------------

In [6]:
import pandas as pd

def t(o):
    # Return the class name of the object
    return o.__class__.__name__

# Use an array to create a Series
series1 = pd.Series([10,20,30])
print "series1 (", t(series1), ')\n', series1
# Notice that the index names were automatically generated as 0,1,2
# Use a dict to create a Series
# notice that the keys of the dict will be the index names
series2 = pd.Series({'row1':11,'row2':22,'row3':33})
print "series2 (", t(series2), ')\n', series2

 series1 ( Series )
0    10
1    20
2    30
dtype: int64
series2 ( Series )
row1    11
row2    22
row3    33
dtype: int64


In [7]:
# Get values from series using the indexes
print series1[0], series2[0], series2['row1']
# Can also use slicing and interesting operations 
# like array in array [[]] to select specific indexes
print series1[1:]
print series1[[0,2]]
print series2['row2':]
print series2[['row1', 'row3']]

10 11 11
1    20
2    30
dtype: int64
0    10
2    30
dtype: int64
row2    22
row3    33
dtype: int64
row1    11
row3    33
dtype: int64


Using DataFrames
----------------

In [8]:
# Create a DataFrame using a two-dimensional array
# Notice that the indexes and column names were automatically generated
df1 = pd.DataFrame([[10,20,30], [40,50,60]])
print "Dataframe from array: df1(", t(df1), ')'
print df1

# Use a dict to give names to 
df2 = pd.DataFrame([{'col1':10,'col2':20,'col3':30}, {'col1':40,'col2':50,'col3':60}])
print "Dataframe from dict: df2(", t(df2), ')'
print df2

# Give names to indexes
df3 = pd.DataFrame([
    {'col1':10,'col2':20,'col3':30}, 
    {'col1':40,'col2':50,'col3':60}
], index=['idx1', 'idx2'])
print "Dataframe from dict, named indexes: df3(", t(df3), ')'
print df3

# What happens when columns are missing
df4 = pd.DataFrame([{'col1':10,'col2':20,'col3':30}, {'col2':40,'col3':50,'col4':60}])
print "Dataframe from dict, missing columns: df4(", t(df4), ')'
print df4

# Create a DataFrame by combining series
df5 = pd.DataFrame([pd.Series([1,2]), pd.Series([3,4])], index=['a', 'b'], )
print "Dataframe from series: df5(", t(df5), ')'
print df5

Dataframe from array: df1( DataFrame )
    0   1   2
0  10  20  30
1  40  50  60
Dataframe from dict: df2( DataFrame )
   col1  col2  col3
0    10    20    30
1    40    50    60
Dataframe from dict, named indexes: df3( DataFrame )
      col1  col2  col3
idx1    10    20    30
idx2    40    50    60
Dataframe from dict, missing columns: df4( DataFrame )
   col1  col2  col3  col4
0  10.0    20    30   NaN
1   NaN    40    50  60.0
Dataframe from series: df5( DataFrame )
   0  1
a  1  2
b  3  4


In [9]:
print "df3(", t(df3), ")\n", df3

# Pick values from a dataframe using array indexing
print "Get value\n", df3['col2']['idx2']

# We can get a column as a series
print "Get column as series\n", df3['col3']

# Or multiple columns (as a DatFrame)
print "Get column as series\n", df3[['col3', 'col2']]

# We can also get the column by its idx
print "Get column by index\n", df3[df3.columns[2]]

# Pick an index (select horizontal line) as a series
print "Get index as a series\n", df3.loc['idx1']

# Also can pick by index number
print "Get index as a series by index\n", df3.iloc[0]

df3( DataFrame )
      col1  col2  col3
idx1    10    20    30
idx2    40    50    60
Get value
50
Get column as series
idx1    30
idx2    60
Name: col3, dtype: int64
Get column as series
      col3  col2
idx1    30    20
idx2    60    50
Get column by index
idx1    30
idx2    60
Name: col3, dtype: int64
Get index as a series
col1    10
col2    20
col3    30
Name: idx1, dtype: int64
Get index as a series by index
col1    10
col2    20
col3    30
Name: idx1, dtype: int64


Modifying DataFrames
--------------------

In [28]:
# Let's copy because some of the following operators change the dataframes
df = df3.copy()
print df

print "Change values of a column"
df['col1'] = [11,41]
print df

print "Change values of an index"
df.loc['idx1'] = [11,21, 31]
print df

print "Add another column to an existing dataframe (changes DataFrame)"
df['col4'] = [1,2]
print df

print "Add another row (index) to an existing dataframe (changes DataFrame)"
df.loc['idx3']=[100,200,300,400]
print df

print "Drop a row (returns new object)"
print df.drop('idx1')

print "Drop a column (returns new object)"
print df.drop('col1', axis=1)

print "Rename index (returns new object)"
print df.rename(index={'idx1': 'new-idx-1'})

print "Rename column (returns new object)"
print df.rename(columns={'col1': 'new-col-1'})

print "Transpose array- change columns to rows and vice versa"
print df.T

print "Double transpose - returns the initial DataFrame"
print df.T.T

      col1  col2  col3
idx1    10    20    30
idx2    40    50    60
Change values of a column
      col1  col2  col3
idx1    11    20    30
idx2    41    50    60
Change values of an index
      col1  col2  col3
idx1    11    21    31
idx2    41    50    60
Add another column to an existing dataframe (changes DataFrame)
      col1  col2  col3  col4
idx1    11    21    31     1
idx2    41    50    60     2
Add another row (index) to an existing dataframe (changes DataFrame)
      col1  col2  col3  col4
idx1    11    21    31     1
idx2    41    50    60     2
idx3   100   200   300   400
Drop a row (returns new object)
      col1  col2  col3  col4
idx2    41    50    60     2
idx3   100   200   300   400
Drop a column (returns new object)
      col2  col3  col4
idx1    21    31     1
idx2    50    60     2
idx3   200   300   400
Rename index (returns new object)
           col1  col2  col3  col4
new-idx-1    11    21    31     1
idx2         41    50    60     2
idx3        100   200  

More advanced operations
------------------------

In [27]:
authors_df=pd.DataFrame([{'id': 1, 'name':'Stephen King'}, {'id': 2, 'name':'Michael Crichton'}])
#authors_df.set_index('id', inplace=True)
books_df=pd.DataFrame([
        {'id': 1, 'author_id':1, 'name':'It'}, 
        {'id': 2, 'author_id':1, 'name':'The Stand'}, 
        {'id': 3, 'author_id':2, 'name':'Airframe'},
        {'id': 4, 'author_id':2, 'name':'Jurassic Park'}
    ])
print authors_df
print books_df
print books_df.merge(authors_df, left_on='author_id', right_on='id')

   id              name
0   1      Stephen King
1   2  Michael Crichton
   author_id  id           name
0          1   1             It
1          1   2      The Stand
2          2   3       Airframe
3          2   4  Jurassic Park
   author_id  id_x         name_x  id_y            name_y
0          1     1             It     1      Stephen King
1          1     2      The Stand     1      Stephen King
2          2     3       Airframe     2  Michael Crichton
3          2     4  Jurassic Park     2  Michael Crichton


A real-world example
====================

To continue with a real-world example, I will use [MovieLens 100k](http://grouplens.org/datasets/movielens/) to represent some `pivot_table` operations. To load the data I've used the code already provided by the [three part series I already mentioned](http://www.gregreda.com/2013/10/26/working-with-pandas-dataframes/)

In [318]:
import os
import pandas as pd

path = 'C:/Users/serafeim/Downloads/ml-100k' # Change this to your own directory
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(os.path.join(path, 'u.user'), sep='|', names=u_cols, encoding='latin-1')
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(os.path.join(path, 'u.data'), sep='\t', names=r_cols, encoding='latin-1')
m_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']
movies = pd.read_csv(os.path.join(path, 'u.item'), sep='|', names=m_cols, usecols=range(5), encoding='latin-1')
movie_ratings = pd.merge(movies, ratings)
lens = pd.merge(movie_ratings, users)

In [99]:
lens['movie_id']

0           1
1           4
2           5
3           7
4           8
5           9
6          11
7          12
8          15
9          17
10         19
11         21
12         22
13         23
14         24
15         25
16         28
17         30
18         31
19         32
20         42
21         44
22         45
23         47
24         48
25         49
26         50
27         54
28         55
29         56
         ... 
99970     332
99971     334
99972     338
99973     346
99974     682
99975     873
99976     877
99977     886
99978    1527
99979     272
99980     288
99981     294
99982     300
99983     310
99984     313
99985     322
99986     328
99987     333
99988     338
99989     346
99990     354
99991     362
99992     683
99993     689
99994     690
99995     748
99996     751
99997     879
99998     894
99999     901
Name: movie_id, dtype: int64

In [6]:
droplens = lens.drop(['video_release_date', 'imdb_url'], 1)
droplens['release_year']=droplens['release_date'].apply(lambda x: str(x).split('-')[2] if len(str(x).split('-'))>2 else 0)
droplens.head()

,movie_id,title,release_date,user_id,rating,unix_timestamp,age,sex,occupation,zip_code,release_year
0,1,Toy Story (1995),01-Jan-1995,308,4,887736532,60,M,retired,95076,1995
1,4,Get Shorty (1995),01-Jan-1995,308,5,887737890,60,M,retired,95076,1995
2,5,Copycat (1995),01-Jan-1995,308,4,887739608,60,M,retired,95076,1995
3,7,Twelve Monkeys (1995),01-Jan-1995,308,4,887738847,60,M,retired,95076,1995
4,8,Babe (1995),01-Jan-1995,308,5,887736696,60,M,retired,95076,1995


In [72]:
droplens[droplens['release_year'] > '1995'].pivot_table(
    columns=['rating', 'sex',],
    index=[ 'release_year'],
    values=[ 'age'], 
    fill_value=0,
    margins=True 
)


age                                              \
rating                1                     2                     3   
sex                   F          M          F          M          F   
release_year                                                          
1996          30.486239  30.373198  32.781437  31.576959  33.086124   
1997          30.333333  30.795053  31.926647  33.264409  32.638767   
1998          25.105263  31.028169  34.205882  36.059524  32.360000   
All           30.181609  30.593199  32.400000  32.404891  32.857241   

                                                                                
rating                           4                     5                   All  
sex                   M          F          M          F          M             
release_year                                                                    
1996          32.718188  32.757335  32.178464  30.494777  32.073340  32.126852  
1997          33.808467  33.224982  33.826004  32.225541  33.025606  33.086874  
1998          34.153061  32.845238  33.860656  33.785714  34.089431  33.403633  
All           33.208988  32.970278  32.986571  31.375121  32.611830  32.598420

In [178]:
droplens[droplens['release_year'] > '1995'].pivot_table(index=['rating'], values=['age'], columns=['sex'] ).dtypes

     sex
age  F      float64
     M      float64
dtype: object

How to use the pivot!

In [29]:
droplens[droplens['release_year'] > '1995'].head().pivot(index='movie_id', columns='age', values='rating')

NameError: name 'droplens' is not defined